In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

import pandas as pd
import numpy as np
import os

from config import *

In [24]:
class IntentDataset(Dataset):

  def __init__(self, csv_path, tokenizer, classes=None):
    df = pd.read_csv(csv_path, sep=";")
    if classes is None:
        self.classes = sorted(df.intent.unique())
    else:
        self.classes = classes

    self.id2class = {i: x for i, x in enumerate(self.classes)}
    self.class2id = {x: i for i, x in enumerate(self.classes)}

    self.labels = [self.class2id[x] for x in df.intent]
    self.objects = list(df.text)
    self.num_classes = len(self.classes)

    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.objects)

  def __getitem__(self, i):
    text = str(self.objects[i])

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

    return {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(self.labels[i], dtype=torch.long)
    }

In [25]:
epochs = 50

tokenizer = BertTokenizer.from_pretrained("cointegrated/rubert-tiny")

train_set = IntentDataset(os.path.join(classification_dataset, "train.csv"), tokenizer)
train_loader = DataLoader(train_set, batch_size=3, shuffle=True)

valid_set = IntentDataset(os.path.join(classification_dataset, "test.csv"), tokenizer, train_set.classes)
valid_loader = DataLoader(valid_set, batch_size=1)


model = BertForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=train_set.num_classes)
model = model.cuda()
model = model.train()

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
loss_function = torch.nn.CrossEntropyLoss()

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [26]:
train_set.num_classes, len(train_set)

(8, 49)

In [14]:
def train_epoch(model, device='cuda'):
    losses = []
    correct_predictions = 0

    for data in train_loader:
        input_ids = data["input_ids"].to(device)
        attention_mask = data["attention_mask"].to(device)
        targets = data["targets"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
            )

        preds = torch.argmax(outputs.logits, dim=1)
        loss = loss_function(outputs.logits, targets)

        correct_predictions += torch.sum(preds == targets)

        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    train_acc = correct_predictions.double() / len(train_set)
    train_loss = np.mean(losses)
    return train_acc, train_loss


def eval_epoch(model, device='cuda'):
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for data in valid_loader:
            input_ids = data["input_ids"].to(device)
            attention_mask = data["attention_mask"].to(device)
            targets = data["targets"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
                )

            preds = torch.argmax(outputs.logits, dim=1)
            loss = loss_function(outputs.logits, targets)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    val_acc = correct_predictions.double() / len(valid_set)
    val_loss = np.mean(losses)
    return val_acc, val_loss

In [15]:
best_accuracy = 0
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    model = model.train()
    train_acc, train_loss = train_epoch(model)
    print(f'Train loss {train_loss} accuracy {train_acc}')

    model = model.eval()
    val_acc, val_loss = eval_epoch(model)
    print(f'Val loss {val_loss} accuracy {val_acc}')
    print('-' * 10)

Epoch 1/50
Train loss 2.0516065429238712 accuracy 0.22448979591836732
Val loss 1.966776818037033 accuracy 0.25
----------
Epoch 2/50
Train loss 1.8335944063523237 accuracy 0.5102040816326531
Val loss 1.8056159019470215 accuracy 0.25
----------
Epoch 3/50
Train loss 1.5870801561019 accuracy 0.5918367346938775
Val loss 1.6328246891498566 accuracy 0.25
----------
Epoch 4/50
Train loss 1.309303458999185 accuracy 0.7551020408163265
Val loss 1.4723696410655975 accuracy 0.625
----------
Epoch 5/50
Train loss 1.1292449937147253 accuracy 0.8571428571428571
Val loss 1.2839204296469688 accuracy 0.75
----------
Epoch 6/50
Train loss 0.9249447619213778 accuracy 0.9387755102040816
Val loss 1.1286193877458572 accuracy 0.75
----------
Epoch 7/50
Train loss 0.784454065210679 accuracy 0.9591836734693877
Val loss 0.9912850707769394 accuracy 0.75
----------
Epoch 8/50
Train loss 0.6463715749628404 accuracy 0.9999999999999999
Val loss 0.8667638227343559 accuracy 0.875
----------
Epoch 9/50
Train loss 0.549

In [16]:
model = model.eval()
model.id2class = train_set.id2class
model.tokenizer = tokenizer

def inference(text, device='cuda'):
    encoding = model.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)
    with torch.no_grad():
        out = model(input_ids=input_ids, attention_mask=attention_mask).logits.detach().cpu()

    return model.id2class[torch.argmax(out, dim=1).item()]


In [17]:
for i in valid_set:
    print(i['text'], inference(i['text']))

Нужно сделать аринжировку arrangement
У меня есть тексти и ноты, можно у вас записать вокал? recording
Хотим приятный дизайн обложки album_cover
Хотим чтобы новый трек был на радио promotion
Есть хорошая демка, хочу в скором времени выпустить её на спотифай release
Почему так дорого trade
Добрый день other
Как тебя зовут? name


In [18]:
torch.save(model, r"weights/best.pt")